In [1]:
import os

# Set a new working directory
new_path = '../'
os.chdir(new_path)

In [2]:
from morphomics import protocols, utils
import tomli

# Path to the toml file. Contains the parameters for each protocol
parameters_filepath = "examples/Parameter_files/Morphomics.Parameters_trial.toml"

# read the toml file
with open(parameters_filepath, mode="rb") as _parameter_file:
    parameters = tomli.load(_parameter_file)

ModuleNotFoundError: No module named 'morphomics.Topology'

In [ ]:
#print the parameters for each protocol
for key in list(parameters.keys()):
    print(key, '::', parameters[key])

In [ ]:
# get the protocol
# from already saved protocol
# or initialize a new protocol with parameters
if parameters["load_previous_instance"]:
    last_instance_path = os.path.join(parameters["path_to_last_instance"], f'last_instance_{parameters["Parameters_ID"]}')
    protocol = utils.load_obj(last_instance_path)
else:
    protocol = protocols.Protocols(parameters, parameters["Parameters_ID"])

In [ ]:
script_sequence = parameters["Protocols"]
print(script_sequence)

In [ ]:
# input
perform_input = getattr(protocol, 'Input')
perform_input()

In [ ]:
tmd_microglia = protocol.morphoframe['TMD']
print(tmd_microglia.shape)

In [ ]:
import tmd
sample = tmd_microglia['Barcodes'][0]
tmd.plot.barcode(sample)

In [ ]:
# clean frame
perform_input = getattr(protocol, 'Clean_frame')
perform_input()

In [ ]:
# load cleaned input

parameters['Protocols'].append('Load_data')
parameters['Load_data'] = {'folderpath_to_data': '/home/thomas/Desktop/morphOMICs/examples/trial_folder/tmd_cleaned/Morphomics.PID_trial.Cleaned',
                           'morphoframe_name': 'TMD'}# 
perform_input = getattr(protocol, 'Load_data')
perform_input()

In [ ]:
protocol.morphoframe

In [ ]:
df = protocol.morphoframe['TMD']

In [ ]:
# PI
perform_input = getattr(protocol, 'Persistence_Images')
perform_input()

In [ ]:
pis = protocol.metadata['PI_matrix']

In [ ]:
pis.shape

In [ ]:
df['pi'] = [row for row in pis]


In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np

# Create a random DataFrame similar to your data
np.random.seed(42)  # For reproducibility
num_rows = 1175
num_cols = 10
df = pd.DataFrame(np.random.rand(num_rows, num_cols), columns=[f'col_{i}' for i in range(num_cols)])

# Add a 'Model' column with four different values
df['Model'] = np.random.choice(['A', 'B', 'C', 'D'], size=num_rows)

# Create a random numpy array to represent 'pis'
pis = np.random.rand(num_rows, 400)

# Add the 'pi' column to the DataFrame
df['pi'] = [pi for pi in pis]

# Function to perform bootstrapping
def bootstrap_mean(data):
    # Perform bootstrapping by sampling with replacement
    bootstrapped_means = []
    num_samples = 500
    sample_size = 15
    for _ in range(num_samples):
        sample = np.random.choice(data, size=sample_size, replace=True)
        bootstrapped_means.append(np.mean(sample))
    return bootstrapped_means

# Perform bootstrapping by Model
unique_models = df['Model'].unique()
bootstrapped_means_by_model = {}
for model in unique_models:
    model_data = df[df['Model'] == model]['pi']
    bootstrapped_means_by_model[model] = bootstrap_mean(model_data)

# Create a new DataFrame to store the bootstrapped means
bootstrapped_df = pd.DataFrame({
    'Model': np.repeat(unique_models, 500),  # 500 bootstrapped samples for each model
    'Bootstrapped_Mean': np.concatenate([np.array(arr).flatten() for arr in bootstrapped_means_by_model.values()])
})

# Print or display the bootstrapped DataFrame
print(bootstrapped_df.head())


In [ ]:
# UMAP
perform_input = getattr(protocol, 'UMAP')
perform_input()

In [ ]:
# reduction
perform_input = getattr(protocol, 'Prepare_ReductionInfo')
perform_input()

In [ ]:
# Plotting
perform_input = getattr(protocol, 'Plotting')
perform_input()

In [ ]:
for sequence in script_sequence:
    print("Doing %s..."%sequence)
    perform_this = getattr(protocol, sequence)
    perform_this()

if parameters["save_last_instance"]:
    new_instance_path = os.path.join(parameters["path_to_new_instance"], f'last_instance_{parameters["Parameters_ID"]}')
    utils.save_obj(protocol, last_instance_path)

In [ ]:
protocol.morphoframe.keys()

In [ ]:
protocol.morphoframe['bootstrap_frame']

In [ ]:
protocol.morphoframe['morphoframe']

In [ ]:
protocol.metadata.keys()

In [ ]:
protocol.metadata['bootstrap_info']

In [ ]:
protocol.metadata['PI_matrix'].shape